In [1]:
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings('ignore') 


# PHẦN 1: CÁC HÀM TIỀN XỬ LÝ SỐ LIỆU THÔ

def clean_number(x):
    """Xử lý triệt để các định dạng: 1.75M, 396.57K, 5,500.00, #######"""
    if pd.isna(x):
        return np.nan
    x_str = str(x).strip().upper()       
    x_str = x_str.replace(',', '') # Bỏ dấu phẩy phân cách ngàn
    
    try:
        if 'M' in x_str:
            return float(x_str.replace('M', '')) * 1_000_000
        elif 'K' in x_str:
            return float(x_str.replace('K', '')) * 1_000
        elif 'B' in x_str:
            return float(x_str.replace('B', '')) * 1_000_000_000
        else:
            return float(x_str)
    except:
        return np.nan

In [2]:
# PHẦN 2: XỬ LÝ DỮ LIỆU GIÁ TỪ FOLDER -> TÍNH CRASH RISK & BIẾN THỊ TRƯỜNG
print("1. Đang tiền xử lý dữ liệu Giá Chứng Khoán (Market Data)...")

path_to_prices = 'Gia_CK' # <-- Đảm bảo thư mục này nằm cùng chỗ với file .ipynb
all_files = glob.glob(os.path.join(path_to_prices, "*.csv"))

df_market_list = []

for filename in all_files:
    try:
        # Lấy Ticker từ tên file
        ticker = os.path.basename(filename).replace('.csv', '').split(' ')[-1]
        
        df = pd.read_csv(filename)
        # Đổi tên cột
        df.rename(columns={'Ngày': 'Date', 'Lần cuối': 'Close', 'KL': 'Volume'}, inplace=True)
        
        # Chỉ giữ lại Date, Close, Volume
        df = df[['Date', 'Close', 'Volume']]
        
        # Tiền xử lý dữ liệu
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')
        df['Close'] = df['Close'].apply(clean_number)
        df['Volume'] = df['Volume'].apply(clean_number)
        
        # Lọc bỏ giá trị NaN
        df.dropna(subset=['Date', 'Close'], inplace=True)
        
        # Lọc giai đoạn 2021 - 2024
        df = df[(df['Date'].dt.year >= 2021) & (df['Date'].dt.year <= 2024)]
        
        # LỌC MÃ KHÔNG ĐỦ NĂM: Xét xem có tồn tại dữ liệu năm 2021 và 2024 không
        years_present = df['Date'].dt.year.unique()
        if 2021 not in years_present or 2024 not in years_present:
            continue # Bỏ qua mã này
            
        df.sort_values('Date', inplace=True)
        df.set_index('Date', inplace=True)
        
        # TÍNH LỢI SUẤT TUẦN (Weekly Returns) & KHỐI LƯỢNG TUẦN
        # Resample lấy giá đóng cửa ngày thứ 6, và tổng khối lượng cả tuần
        weekly_df = pd.DataFrame()
        weekly_df['Close'] = df['Close'].resample('W-FRI').last()
        weekly_df['Volume'] = df['Volume'].resample('W-FRI').sum()
        
        weekly_df['W_Ret'] = weekly_df['Close'].pct_change()
        
        weekly_df = weekly_df.reset_index()
        weekly_df['Ticker'] = ticker
        weekly_df['Year'] = weekly_df['Date'].dt.year
        weekly_df['Quarter'] = weekly_df['Date'].dt.quarter
        
        df_market_list.append(weekly_df)
        
    except Exception as e:
        pass # Bỏ qua file lỗi ngầm định

df_weekly = pd.concat(df_market_list, ignore_index=True)

# HÀM TÍNH TOÁN CÁC BIẾN Y VÀ THỊ TRƯỜNG THEO QUÝ
def compute_quarterly_market_vars(group):
    rets = group['W_Ret'].dropna()
    vols = group['Volume'].dropna()
    n = len(rets)
    
    if n < 8: # Quý có ít hơn 8 tuần giao dịch -> Dữ liệu rác, bỏ
        return pd.Series([np.nan]*6, index=['NCSKEW', 'DUVOL', 'Sigma', 'Ret', 'Dturn', 'End_Price'])
        
    # 1. Biến Y: NCSKEW & DUVOL
    skewness = rets.skew()
    ncskew = -skewness # NCSKEW cơ bản = - Hệ số bất đối xứng
    
    mean_ret = rets.mean()
    down_rets = rets[rets < mean_ret]
    up_rets = rets[rets >= mean_ret]
    
    if len(down_rets) < 2 or len(up_rets) < 2:
        duvol = np.nan
    else:
        var_down = down_rets.var()
        var_up = up_rets.var()
        # Công thức DUVOL chuẩn
        duvol = np.log(((len(up_rets)-1) * var_down) / ((len(down_rets)-1) * var_up))
        
    # 2. Biến Kiểm soát: Sigma, Ret
    sigma = rets.std()
    ret = mean_ret
    
    # 3. Biến Dturn (Khử xu hướng vòng quay)
    # Vì thiếu số lượng CP lưu hành, dùng Log(Mean Volume) làm proxy, sau đó sẽ detrend toàn bảng
    avg_vol = vols.mean()
    turnover_proxy = np.log(avg_vol) if avg_vol > 0 else np.nan
    
    # Giá đóng cửa cuối quý (Dùng để tính Book-to-Market sau này)
    end_price = group['Close'].iloc[-1]
    
    return pd.Series([ncskew, duvol, sigma, ret, turnover_proxy, end_price], 
                     index=['NCSKEW', 'DUVOL', 'Sigma', 'Ret', 'Turnover_Proxy', 'End_Price'])

print("   Đang tính toán NCSKEW, DUVOL và gom nhóm theo Quý...")
df_market_q = df_weekly.groupby(['Ticker', 'Year', 'Quarter']).apply(compute_quarterly_market_vars).reset_index()

# Khử xu hướng Turnover (Dturn) = Turnover quý t - Trung bình Turnover các quý trước đó
df_market_q['Dturn'] = df_market_q.groupby('Ticker')['Turnover_Proxy'].diff()
df_market_q.drop(columns=['Turnover_Proxy'], inplace=True)


1. Đang tiền xử lý dữ liệu Giá Chứng Khoán (Market Data)...
   Đang tính toán NCSKEW, DUVOL và gom nhóm theo Quý...


In [4]:

# PHẦN 3: XỬ LÝ BÁO CÁO TÀI CHÍNH & TẠO ĐỘ TRỄ (LAGGING)

print("2. Đang xử lý Báo cáo tài chính (Financials_Final.xlsx)...")

df_fin = pd.read_excel('Financials_Final.xlsx')

# Lọc Xử lý duplicates
df_fin.drop_duplicates(subset=['Ticker', 'Year', 'Quarter'], keep='last', inplace=True)

# TÍNH CÁC BIẾN KIỂM SOÁT TÀI CHÍNH
df_fin['Size'] = np.log(df_fin['TotalAssets'] + 1)
df_fin['Lev'] = df_fin['TotalDebt'] / df_fin['TotalAssets']
df_fin['ROA'] = df_fin['NetProfit'] / df_fin['TotalAssets']

# Tính Growth (Tăng trưởng YoY)
df_fin.sort_values(['Ticker', 'Year', 'Quarter'], inplace=True)
df_fin['Rev_Lag4'] = df_fin.groupby('Ticker')['NetRevenue'].shift(4)
df_fin['Growth'] = (df_fin['NetRevenue'] - df_fin['Rev_Lag4']) / (df_fin['Rev_Lag4'].abs() + 1)

# Các biến chưa kịp add (sẽ bổ sung sau)
if 'Cashflow' in df_fin.columns:
    df_fin['Cashflow_Ratio'] = df_fin['Cashflow'] / df_fin['TotalAssets']
if 'FixedAsset' in df_fin.columns:
    df_fin['FIXED'] = df_fin['FixedAsset'] / df_fin['TotalAssets']

# TẠO ĐỘ TRỄ (Tất cả X tài chính phải lùi 1 Quý để giải thích cho Y tương lai)
df_fin['Merge_Quarter'] = df_fin['Quarter'] + 1
df_fin['Merge_Year'] = df_fin['Year']

mask_q5 = df_fin['Merge_Quarter'] == 5
df_fin.loc[mask_q5, 'Merge_Year'] += 1
df_fin.loc[mask_q5, 'Merge_Quarter'] = 1

# Các cột sẽ mang đi ghép
cols_fin = ['Ticker', 'Merge_Year', 'Merge_Quarter', 'Size', 'Lev', 'ROA', 'Growth', 'Equity']
df_fin_ready = df_fin[cols_fin]

2. Đang xử lý Báo cáo tài chính (Financials_Final.xlsx)...


In [5]:

# PHẦN 4: MERGE DỮ LIỆU & TẠO BIẾN DML/Causal Forest (D, W)
print("3. Đang ghép nối các bộ dữ liệu và tính toán biến DiD...")

# 1. Ghép Giá (Y) với Tài chính (X trễ)
final_df = pd.merge(
    df_market_q, df_fin_ready,
    left_on=['Ticker', 'Year', 'Quarter'],
    right_on=['Ticker', 'Merge_Year', 'Merge_Quarter'],
    how='inner' # Chỉ giữ các quý tồn tại song song cả Giá và BCTC
)

# 2. Ghép Biến Quản trị tĩnh
df_static = pd.read_excel('Financial_Reports.xlsx')
# Đổi tên chuẩn theo Model 
df_static.rename(columns={'ShortTerm_Bond': 'Treat', 'Auditor': 'Big4', 'Board_Size': 'Board'}, inplace=True)

final_df = pd.merge(final_df, df_static, on='Ticker', how='left')

# 3. TÍNH CÁC BIẾN NÂNG CAO CÒN LẠI
# Tính Book-to-Market (BM): Equity / Proxy Market Cap
final_df['BM'] = final_df['Equity'] / (final_df['End_Price'] * 1000) # (Chia 1000 tuỳ scale tiền)

# Biến Post (Nghị định 08 từ Q2/2023)
final_df['Post'] = np.where((final_df['Year'] > 2023) | ((final_df['Year'] == 2023) & (final_df['Quarter'] >= 2)), 1, 0)

# Biến D (Biến Xử lý cho Double Machine Learning)
final_df['D'] = final_df['Treat'] * final_df['Post']

# Biến Moderator (RISK_LV): Khẩu vị rủi ro đòn bẩy
# Doanh nghiệp có Lev > Trung vị của toàn thị trường trong quý đó = 1, ngược lại = 0
median_lev_per_quarter = final_df.groupby(['Year', 'Quarter'])['Lev'].transform('median')
final_df['RISK_LV'] = np.where(final_df['Lev'] > median_lev_per_quarter, 1, 0)


3. Đang ghép nối các bộ dữ liệu và tính toán biến DiD...


In [6]:
# PHẦN 5: CLEANING CUỐI CÙNG & XUẤT FILE CHO DML
print("4. Xử lý Missing values, Infinity và xuất file...")

# Bỏ các cột rác sinh ra trong quá trình tính
final_df.drop(columns=['Merge_Year', 'Merge_Quarter', 'End_Price', 'Equity'], inplace=True, errors='ignore')

# Xử lý các giá trị vô cực (Infinity) sinh ra từ Log/Chia số 0
final_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Lọc bỏ các dòng thiếu biến Y hoặc biến D (Bắt buộc phải có để chạy DML)
final_df.dropna(subset=['NCSKEW', 'DUVOL', 'D', 'Size', 'Lev'], inplace=True)

# Sắp xếp gọn gàng theo Ticker và Thời gian
final_df.sort_values(['Ticker', 'Year', 'Quarter'], inplace=True)

# Lưu thành CSV và Excel để chạy mô hình
output_file = 'Data_Panel_DML_Ready.xlsx'
final_df.to_excel(output_file, index=False)
final_df.to_csv('Data_Panel_DML_Ready.csv', index=False) 

print(f"\n XONG! DỮ LIỆU ĐÃ SẴN SÀNG.")
print(f"Tổng số quan sát (Dòng): {len(final_df)}")
print(f"Các biến đã có trong file: {list(final_df.columns)}")

4. Xử lý Missing values, Infinity và xuất file...

 XONG! DỮ LIỆU ĐÃ SẴN SÀNG.
Tổng số quan sát (Dòng): 843
Các biến đã có trong file: ['Ticker', 'Year', 'Quarter', 'NCSKEW', 'DUVOL', 'Sigma', 'Ret', 'Dturn', 'Size', 'Lev', 'ROA', 'Growth', 'Treat', 'Big4', 'Board', 'State_Owner', 'BM', 'Post', 'D', 'RISK_LV']
